In [1]:
import json
import pandas as pd
import cmath
import xml.etree.ElementTree as et
import numpy as np
from transformations import *


In [2]:
pd.set_option('display.max_columns', 5000)
#pd.set_option('display.max_rows', None)

In [3]:
def getformations(dataframe): 
    df=dataframe['action_name']
    dft=dataframe['team_name']
    list=[]
    team=[]
    for i in range(4):
        x=df.iloc[i]
        teams = dft.iloc[i]
        if ('2'<x[0]<'6'):
            list+=[x]
            team+=[teams]

    team1 = [team[0], list[0]]
    team2 = [team[1], list[1]]
            
    return team1, team2

In [4]:
#instatiate
template_path='template.csv'
df=pd.read_csv(template_path)
df

,id,matchId,matchPeriod,minute,second,matchTimestamp,videoTimestamp,relatedEventId,shot,groundDuel,aerialDuel,infraction,carry,type.primary,type.secondary,location.x,location.y,team.id,team.name,team.formation,opponentTeam.id,opponentTeam.name,opponentTeam.formation,player.id,player.name,player.position,pass.accurate,pass.angle,pass.height,pass.length,pass.recipient.id,pass.recipient.name,pass.recipient.position,pass.endLocation.x,pass.endLocation.y,possession.id,possession.duration,possession.types,possession.eventsNumber,possession.eventIndex,possession.startLocation.x,possession.startLocation.y,possession.endLocation.x,possession.endLocation.y,possession.team.id,possession.team.name,possession.team.formation,possession.attack,pass,groundDuel.opponent.id,groundDuel.opponent.name,groundDuel.opponent.position,groundDuel.duelType,groundDuel.keptPossession,groundDuel.progressedWithBall,groundDuel.stoppedProgress,groundDuel.recoveredPossession,groundDuel.takeOn,groundDuel.side,groundDuel.relatedDuelId,infraction.yellowCard,infraction.redCard,infraction.type,infraction.opponent.id,infraction.opponent.name,infraction.opponent.position,possession.attack.withShot,possession.attack.withShotOnGoal,possession.attack.withGoal,possession.attack.flank,possession.attack.xg,carry.progression,carry.endLocation.x,carry.endLocation.y,shot.bodyPart,shot.isGoal,shot.onTarget,shot.goalZone,shot.xg,shot.postShotXg,shot.goalkeeperActionId,shot.goalkeeper.id,shot.goalkeeper.name,possession,aerialDuel.opponent.id,aerialDuel.opponent.name,aerialDuel.opponent.position,aerialDuel.opponent.height,aerialDuel.firstTouch,aerialDuel.height,aerialDuel.relatedDuelId,shot.goalkeeper,infraction.opponent,location


In [5]:
#set all unknown attributes
id=matchId=videoTimestamp=relatedEventId=teamId=opponentTeamId=playerId=passrecipientId=possessionId=possessioneventsNumber = 0
possessioneventIndex=possessionteamId=groundDuelopponentId=groundDuelrelatedDuelId=infractionopponentId=shotgoalkeeperActionId=shotgoalkeeperId=aerialDuelopponentId=aerialDuelrelatedDuelId = 0
passheight = None
shotxg=shotgoalzone=possessionattackxg=aerialduelheight=aerialFirsttouch=aerialdueloppheight=np.nan


In [6]:
#load instat file
instat_path = "data/InStat_AXA_Super_2023-06-02_Switzerland. AXA Women's Super League_Servette Chenois_vs_Zurich_2582307_2023-06-02_Servette Chenois_vs_Zurich_2582307_rawdata - Kopie.xml"
instatdf=pd.read_xml(instat_path)
instatdf=instatdf.iloc[2:,:]
instatdf.drop(columns=['column','number','dl', 'id', 'uid', 'action_id', 'player_id', 'team_id', 'standart_id', 'ts', 'position_id', 'opponent_id', 'opponent_team_id','opponent_position_id', 'zone_id', 'zone_dest_id', 'possession_id', 'possession_team_id', 'possession_team_id', 'attack_status_id', 'attack_type_id', 'attack_team_id', 'body_id' ], inplace=True)
mask = (instatdf['second']==0.00) & (~instatdf['player_name'].isna()) & (instatdf['action_name']!='Attacking pass accurate')
instatdf = instatdf[~mask]
todrop = ['line-up', 'Substitute player', '1st half','2nd half', 'Players, that created offside trap']
mask = ~instatdf['action_name'].isin(todrop)
instatdf = instatdf[mask]
teamA,teamB=getformations(instatdf)

In [8]:
teamA, teamB

(['Servette Chenois', '4-2-3-1'], ['Zurich', '4-4-2 classic'])

In [ ]:
instatdf

In [7]:
mask = (instatdf['second']==0)&(instatdf['action_name']!='Attacking pass accurate')
instatdf = instatdf[~mask] 
#ready for wyscout transformation

In [ ]:
#set the first possession length

In [ ]:
#get the index for the new event
def next_event(df, index): 
    ind = index
    return ind

In [10]:
position_transform('Defender - Central', teamA[1])

'CB'

In [25]:
x="Offside"
x=='Offside'

True

In [8]:
current_possession,poss_types, withshot,withshotongoal, withgoal, flank=setnewpossession(instatdf,0) #start x, y, end x, y, possession_length, possessionstart index|possession types, attack variables


In [11]:
flank

nan

In [23]:
(None in [1])

False

In [12]:
def create_event(instat, ind, wyscout):
    index_instat, typeprimary, typesecondary = get_event_type(instat, ind)
    #setup every attribute
    minute, second, matchTimestamp = get_time(instat, ind)
    locx, locy = get_location(instat, ind)
    teamname = instat['team_name'].iloc[ind]
    teamformation = teamA[1] if (teamname==teamA[0]) else teamB[1]
    oppteamname = teamB[0] if (teamname==teamA[0]) else teamA[0]
    oppteamformation = teamB[1] if (teamname==teamA[0]) else teamA[1]
    playername = instat['player_name'].iloc[ind]
    playerposition = position_transform(instat['position_name'].iloc[ind], teamformation)
    passlength = instat['len'].iloc[ind]
    possessionlength = instat['possession_time'].iloc[ind]
    possteamname = instat['possession_team_name'].iloc[ind]
    possteamformation = teamA[1] if (possteamname==teamA[0]) else teamB[1]

    if (current_possession[5]>ind):
        poss_x=poss_y=endposs_x=endposs_y=np.nan
    else:
        poss_x=current_possession[0]
        poss_y=current_possession[1]
        endposs_x=current_possession[2]
        endposs_y=current_possession[3]
    
    if (possessionlength>0):
        current_possession,poss_types, withshot,withshotongoal, withgoal, flank = setnewpossession(instat, ind+1) 
    
    
    new_event = {
        "id": id,
        "matchId": matchId,
        "matchPeriod":get_period(instat,ind),
        "minute":minute,
        "second":second,
        "matchTimestamp":matchTimestamp,
        "videoTimestamp":videoTimestamp,
        "relatedEventId":relatedEventId,
        "shot":1,
        "groundDuel":1,
        "aerialDuel":1,
        "infraction":1,
        "carry":1,
        "type.primary":1,	#
        "type.secondary":1,#
        "location.x":locx,	
        "location.y":locy,	
        "team.id":teamId,	
        "team.name": teamname,	
        "team.formation":teamformation,	
        "opponentTeam.id":opponentTeamId,
        "opponentTeam.name":oppteamname,	
        "opponentTeam.formation":oppteamformation,	
        "player.id":playerId,	
        "player.name":playername,	
        "player.position":playerposition,	
        "pass.accurate":1,	#
        "pass.angle":1,	#
        "pass.height":passheight,	
        "pass.length":passlength,	
        "pass.recipient.id":passrecipientId,	
        "pass.recipient.name":1,	#
        "pass.recipient.position":1,#
        "pass.endLocation.x":1,	#
        "pass.endLocation.y":1,	#
        "possession.id":possessionId,	
        "possession.duration":current_possession[4],	
        "possession.types":poss_types,	
        "possession.eventsNumber":possessioneventsNumber,	
        "possession.eventIndex":possessioneventIndex,	
        "possession.startLocation.x":poss_x,
        "possession.startLocation.y":poss_y,	
        "possession.endLocation.x":endposs_x,	
        "possession.endLocation.y":endposs_y,	
        "possession.team.id":possessionteamId,	
        "possession.team.name":possteamname,	
        "possession.team.formation":possteamformation,	
        "possession.attack":1,	
        "pass":1,	
        "groundDuel.opponent.id":groundDuelopponentId,	
        "groundDuel.opponent.name":1,	
        "groundDuel.opponent.position":1,	
        "groundDuel.duelType":1,	
        "groundDuel.keptPossession":1,	
        "groundDuel.progressedWithBall":1,	
        "groundDuel.stoppedProgress":1,	
        "groundDuel.recoveredPossession":1,	
        "groundDuel.takeOn":1,	
        "groundDuel.side":1,	
        "groundDuel.relatedDuelId":groundDuelrelatedDuelId,	
        "infraction.yellowCard":1,	
        "infraction.redCard":1,	
        "infraction.type":1,	
        "infraction.opponent.id":infractionopponentId,	
        "infraction.opponent.name":1,	
        "infraction.opponent.position":1,	
        "possession.attack.withShot":withshot,	
        "possession.attack.withShotOnGoal":withshotongoal,	
        "possession.attack.withGoal":withgoal,	
        "possession.attack.flank":flank,	
        "possession.attack.xg":possessionattackxg,	
        "carry.progression":1,	
        "carry.endLocation.x":1,	
        "carry.endLocation.y":1,	
        "shot.bodyPart":1,	
        "shot.isGoal":1,	
        "shot.onTarget":1,	
        "shot.goalZone":shotgoalzone,	
        "shot.xg": shotxg,	
        "shot.postShotXg":1,	
        "shot.goalkeeperActionId":shotgoalkeeperActionId,	
        "shot.goalkeeper.id":shotgoalkeeperId,	
        "shot.goalkeeper.name":1,	
        "possession":1,	
        "aerialDuel.opponent.id":aerialDuelopponentId,	
        "aerialDuel.opponent.name":1,	
        "aerialDuel.opponent.position":1,	
        "aerialDuel.opponent.height":aerialdueloppheight,	
        "aerialDuel.firstTouch":aerialFirsttouch,	
        "aerialDuel.height":aerialduelheight,	
        "aerialDuel.relatedDuelId":aerialDuelrelatedDuelId,	
        "shot.goalkeeper":1,	
        "infraction.opponent":1,	
        "location":1,
    }
    wdf = pd.concat([wyscout, pd.DataFrame([new_event])], ignore_index=True)
    
    
    matchend=False
    return instat, index_instat, wdf

In [ ]:
a,b,c=create_event(instatdf, 0, df)
c

In [ ]:
matchend=True
index = 0   
while (matchend):
    instatdf, index, df = create_event(instatdf, index, df)
    if(instatdf['action_name'].iloc[index]=='Match end'):
        break
     
    
    





In [ ]:
def parseaction(instat, ind, wyscout):
    newind=ind
    starttime
    return newind

In [ ]:
instatdf